Em algumas semanas relembraremos 5 anos que meu pai faleceu. Ele sempre foi um incentivador das minhas descobertas e estudos, então resolvi lembrar dessa data com um estudo sobre os números da pandemia de COVID-19.

Enquanto estávamos debatendo em sociedade se a COVI-19 era algo mortal ou não, algumas pessoas afirmavam que havia sub apontamento das mortes causadas por covid enquanto outras alertavam que havia um sobre noticiamento, eu pensava em como medir isso tudo. 

Minha conclusão foi simples: depois que isso tudo passar, basta pegar os dados de óbitos no Brasil, antes, durante e depois da pandemia e comparar os números. Bastava esperar...

Baixei os dados de óbitos no site: https://opendatasus.saude.gov.br/dataset/sim

E assim o projeto começou:
1. verificação da estrutura dos dados;
2. análise e seleção dos dados relevantes ao estudo;
3. unificação dos dados e conversão de formato (csv para parquet)
4. carga dos dados brutos em formato parquet e limpeza:
    a. remoção de óbitos por:
    - acidente; 
    - suicídio; 
    - homicídio; 
    - acidente de trabalho; e 
    - ocorrência policial.
5. análise exploratória visual (gráfico da evolução dos óbitos ao longo do tempo):
    - existe um padrão claro anual no comportamento das mortes no período pré-pandêmico:
    - as mortes aumentam com a chegada do inverno e reduzem com a chegada do verão;
6. decomposição da série temporal;
7. modelagem dos dados de 01/01/2016 a 31/12/2019
8. determinação através de análise visual do gráfico de tendencia da decomposição sazonal, do período de 'pandemia' real;
9. predição dos dados de obito para o período de 'pandemia';
10. ajuste dos dados de óbito, fazendo a subtração dos óbitos registrados pelos preditos;
11. cálculo do numero de óbitos da curva ajustada, representando o número de vítimas diretas e indiretas da COVID-19;
12. cálculo do número total de óbitos;
13. proporção entre o total de óbitos e os óbitos ajustados, mostram a proporção das vítimas da pandemia.

Outras informaçãoes:
- OMS declara pandemia de COVID-19: 11/03/2020
- Carnaval 2021: 
    - 15/02/2021;
    - festividades seguiram normalmente;
    - máximo local na curva de mortalidade em 28/02/2024

# Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import warnings
import time
from   datetime import datetime, timedelta
from   tqdm     import tqdm # faz a barra de progresso

# Bibliotecas de manipulação de arquivos
import os
import glob

# Bibliotecas para visualização dos dados
# import plotly.express as px
# import plotly.graph_objects as go
# from   plotly.subplots import make_subplots
import matplotlib.pyplot as plt

from bokeh.plotting import figure, show
from bokeh.models   import ColumnDataSource, HoverTool
from bokeh.layouts  import gridplot
from bokeh.io       import output_notebook
output_notebook(verbose=False)

# Bibliotecas de ML
from statsmodels.graphics.api import qqplot
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.optimize import curve_fit
# import statsmodels.api as sm
import scipy.stats     as stats


import numpy as np
import pandas as pd
# 


Loading BokehJS ...

# Definições Iniciais

In [2]:
base_path = os.path.abspath(os.getcwd()) + '/dataset/'
base_path = base_path.replace('\\', '/')

# Funções

### Formata um número (int ou float) no formato x.xxx,xx

In [3]:
def formatar_numero_br(numero):
    """
    Formata um número (int ou float) para o padrão brasileiro de milhares e decimais.

    Args:
        numero: O número inteiro ou float a ser formatado.

    Returns:
        Uma string com o número formatado no padrão brasileiro.
    """
    if isinstance(numero, (int, float)):
        # Converte o número para string com duas casas decimais, se for float
        if isinstance(numero, float):
            numero_str = f"{numero:.2f}"
            
            # Separa a parte inteira da parte decimal
            partes = numero_str.split('.')
            parte_inteira = partes[0]
            parte_decimal = partes[1]

            # Formata a parte inteira com o separador de milhar '.'
            milhar_formatado = f"{int(parte_inteira):,}".replace(",", ".")
            
            # Recompõe o número com a vírgula como separador decimal
            numero_final = f"{milhar_formatado},{parte_decimal}"
        else: # Se for um número inteiro
            numero_final = f"{numero:,}".replace(",", ".")
        
        return numero_final
    else:
        return "Entrada inválida. Por favor, insira um número inteiro ou float."


### Cria gráfico com Bokeh - com dois eixos y

In [4]:
def grafico_bokeh_duplo(
    df,
    x_col,
    y1_col,
    y2_col, 
    titulo="Ajuste Sazonal aos Dados", 
    y1_legend = "Dados Reais",
    y2_legend = "Ajuste dos dados",
    y1_color="navy",
    y2_color="green",
    notebook=True,
    hachura=False

):
    """
    Plota os dados reais e o ajuste sazonal usando Bokeh, de forma genérica.

    Parâmetros:
        df: DataFrame com as colunas de eixo x (x_col) e y (y_col)
        y_fit: array ou lista com os valores ajustados (mesmo tamanho de df)
        x_col: nome da coluna a ser usada como eixo x
        y_col: nome da coluna a ser usada como eixo y
        titulo: título do gráfico
        notebook: ativa output_notebook se True (padrão)
    """
    if notebook:
        output_notebook()

    df_plot = df.copy()
    source = ColumnDataSource(df_plot)

    # Detectar tipo de eixo x
    x_axis_type = 'datetime' if pd.api.types.is_datetime64_any_dtype(df_plot[x_col]) else 'linear'

    p = figure(
        title=titulo,
        x_axis_label=x_col,
        y_axis_label=y1_col,
        x_axis_type=x_axis_type,
        width=950, height=400,
        tools="pan,wheel_zoom,box_zoom,reset,save"
    )

    # Linha dos dados reais
    p.line(x=x_col, y=y1_col, source=source, legend_label=y1_legend,
           color=y1_color, alpha=0.6, line_width=2)

    # Linha do ajuste
    p.line(x=x_col, y=y2_col, source=source, legend_label=y2_legend,
           color=y2_color, line_width=2)
    if hachura:
        p.varea(x=x_col, y1=y2_col, y2=0, source=source, fill_color="lightblue", alpha=0.5)

    # Hover tool
    hover = HoverTool(
        tooltips=[
            (x_col, f"@{x_col}" + ('{%F}' if x_axis_type == 'datetime' else '')),
            (y2_col, f"@{y2_col}"),
            ("Ajuste", "@Ajuste_Sazonal{0.0}")
        ],
        formatters={f'@{x_col}': 'datetime'} if x_axis_type == 'datetime' else {},
        mode='vline'
    )
    p.add_tools(hover)
    p.legend.location = "top_left"
    p.xaxis.major_label_orientation = 1
    show(p)

### Teste de Normalidade de Resíduos

In [5]:
def verificar_normalidade_residuo(residual):
    """
    Verifica a normalidade do resíduo usando o teste de Shapiro-Wilk,
    Kolmogorov-Smirnov e QQ-plot.
    """
    # print("Teste de Shapiro-Wilk:")
    # shapiro_stat, shapiro_p = stats.shapiro(residual.dropna())
    # print(f"Estatística={shapiro_stat:.4f}, p-valor={shapiro_p:.4f}")
    # if shapiro_p > 0.05:
    #     print("Resíduo provavelmente segue uma distribuição normal (não rejeita H0).")
    # else:
    #     print("Resíduo provavelmente NÃO segue uma distribuição normal (rejeita H0).")
        
    print("\nTeste de Kolmogorov-Smirnov:")
    # Normalizamos os resíduos para o teste KS
    resid_norm = (residual - np.nanmean(residual)) / np.nanstd(residual)
    ks_stat, ks_p = stats.kstest(resid_norm.dropna(), 'norm')
    print(f"Estatística={ks_stat:.4f}, p-valor={ks_p:.4f}")
    if ks_p > 0.05:
        print("Resíduo provavelmente segue uma distribuição normal (não rejeita H0).")
    else:
        print("Resíduo provavelmente NÃO segue uma distribuição normal (rejeita H0).")
    
    print("\nQQ-plot dos resíduos:")
    qqplot_bokeh(residual)


In [6]:
def qqplot_bokeh(residual, title='QQ-plot dos resíduos'):
    """
    Gera um QQ-plot interativo dos resíduos usando Bokeh.
    """
    output_notebook()  # Para Jupyter. Remova se for script puro.

    # Remove NaN
    resid = pd.Series(residual).dropna().values

    # Obtém quantis teóricos e quantis dos dados
    (osm, osr), (slope, intercept, r) = stats.probplot(resid, dist="norm")
    df = pd.DataFrame({'Theoretical Quantiles': osm, 'Ordered Values': osr})

    source = ColumnDataSource(df)

    p = figure(
        width=500, height=300,
        title=title,
        x_axis_label="Quantis teóricos",
        y_axis_label="Valores ordenados",
        tools="pan,wheel_zoom,box_zoom,reset,save"
    )

    # Pontos do QQ-plot
    p.scatter(x='Theoretical Quantiles', y='Ordered Values', source=source, size=7, color="navy", alpha=0.7)

    # Linha teórica de normalidade
    x_line = np.array([osm.min(), osm.max()])
    y_line = slope * x_line + intercept
    p.line(x_line, y_line, color="red", line_width=2, legend_label="Normalidade teórica")

    # Hover Tool
    hover = HoverTool(
        tooltips=[
            ("Quantil teórico", "@{Theoretical Quantiles}{0.2f}"),
            ("Valor ordenado", "@{Ordered Values}{0.2f}")
        ]
    )
    p.add_tools(hover)
    p.legend.location = "top_left"

    show(p)

### Gráfico da decomposição sazonal

In [7]:
def plotar_decomposicao_temporal_bokeh(decomposicao_sazonal, trend, seasonal, resid, width=120, height=900):
    """
    Plota a decomposição da série temporal em quatro gráficos usando Bokeh.
    Espera pandas Series ou DataFrames como entrada.
    """
    output_notebook()  # Retire se não estiver usando Jupyter

    # Converte para DataFrame se necessário
    df = pd.DataFrame({
        'observed': decomposicao_sazonal.observed,
        'trend': trend,
        'seasonal': seasonal,
        'residual': resid
    })
    df['date'] = df.index

    source = ColumnDataSource(df)
    tools = "pan,wheel_zoom,box_zoom,reset,save"

    # Observada
    p1 = figure(
        height=int(height*1.3), width=width, x_axis_type='datetime', tools=tools,
        title='Número diário de óbitos, exceto: ocorrência policial, acidente de trabalho, acidente, suicídio, homicídio'
    )
    p1.line('date', 'observed', source=source, color="navy")
    p1.add_tools(HoverTool(tooltips=[("Data", "@date{%F}"), ("Observado", "@observed")], formatters={'@date': 'datetime'}))
    p1.grid.grid_line_color = "blue"
    p1.grid.grid_line_alpha = 0.1

    # Tendência
    p2 = figure(
        height=height, width=width, x_axis_type='datetime', tools=tools,
        title='Tendência', x_range=p1.x_range
    )
    p2.line('date', 'trend', source=source, color="black")
    p2.add_tools(HoverTool(tooltips=[("Data", "@date{%F}"), ("Tendência", "@trend")], formatters={'@date': 'datetime'}))
    p2.grid.grid_line_color = "blue"
    p2.grid.grid_line_alpha = 0.1

    # Sazonal
    p3 = figure(
        height=height, width=width, x_axis_type='datetime', tools=tools,
        title='Componente sazonal', x_range=p1.x_range
    )
    p3.line('date', 'seasonal', source=source, color="green")
    p3.add_tools(HoverTool(tooltips=[("Data", "@date{%F}"), ("Sazonal", "@seasonal")], formatters={'@date': 'datetime'}))
    p3.grid.grid_line_color = "blue"
    p3.grid.grid_line_alpha = 0.1

    # Resíduo
    p4 = figure(
        height=height, width=width, x_axis_type='datetime', tools=tools,
        title='Resíduo', x_range=p1.x_range
    )
    p4.line('date', 'residual', source=source, color="red")
    p4.add_tools(HoverTool(tooltips=[("Data", "@date{%F}"), ("Resíduo", "@residual")], formatters={'@date': 'datetime'}))
    p4.grid.grid_line_color = "blue"
    p4.grid.grid_line_alpha = 0.1

    grid = gridplot([[p1], [p2], [p3], [p4]], toolbar_location='right')
    show(grid)

In [8]:
def plotar_serie_temporal_obitos_bokeh(df_obitos_diarios, width=900, height=400):
    """
    Gera um gráfico de série temporal dos óbitos diários usando Bokeh (interativo, sem warnings).
    """
    if df_obitos_diarios is not None and not df_obitos_diarios.empty:
        output_notebook()  # Para exibir no Jupyter. Remova se estiver em script standalone.

        df = df_obitos_diarios.copy()
        df['DTOBITO'] = pd.to_datetime(df['DTOBITO'])

        source = ColumnDataSource(df)
        p = figure(title='Série Temporal de Óbitos Diários, exceto: ocorrência policial, acidente de trabalho, acidente, suicídio, homicídio',
                   x_axis_label='Data do Óbito', y_axis_label='Número de Óbitos',
                   x_axis_type='datetime', 
                   width = width, 
                   height= height,

                   tools="pan,wheel_zoom,box_zoom,reset,save")

        p.line('DTOBITO', 'NUM_OBITOS', source=source, line_width=2, legend_label="Óbitos")
    
        hover = HoverTool(
            tooltips=[
                ("Data", "@DTOBITO{%F}"),
                ("Óbitos", "@NUM_OBITOS"),
            ],
            formatters={'@DTOBITO': 'datetime'},
            mode='vline'
        )
        p.add_tools(hover)
        p.legend.location = "top_left"
        show(p)
    else:
        print("Não há dados para gerar o gráfico de série temporal.")


# Junta os bancos e reduz os dados para os necessários

Foi desabilitado pois os dados já foram convertidos anteriormente

In [9]:
# cols = ['ORIGEM', 'TIPOBITO', 'DTOBITO', 'DTNASC',
#  'IDADE', 'SEXO', 'RACACOR', 'LOCOCOR', 'CIRURGIA',
#  'CAUSABAS', 'CB_PRE', 'CIRCOBITO', 'ACIDTRAB', 'FONTE',
#  'CAUSABAS_O', 'TPOBITOCOR', 'FONTES', 'CONTADOR']

In [10]:
# in_file = 'Mortalidade_Geral_2024.csv'
# # in_file = 'teste.csv'
# parquet_file = in_file[0:len(in_file)-4] + '_resumido.parquet'
# display(in_file)
# parquet_file


In [11]:
# if os.path.exists(base_path+parquet_file):
#     df = pd.read_parquet(base_path+parquet_file, engine='pyarrow')
#     # Converte a coluna DTOBITO para o formato dd/mm/aaaa
#     df['DTOBITO'] = pd.to_datetime(df['DTOBITO'], format='%d%m%Y', errors='coerce')
#     df['DTNASC'] = pd.to_datetime(df['DTNASC'], format='%d%m%Y', errors='coerce')
#     print('Arquivo parquet localizado.')
# else:
#     print('Arquivo csv importado, tratado e salvo como parquet')
#     # Lê o arquivo normalmente, mas sem cabeçalho
#     df = pd.read_csv(base_path + in_file, sep=';', quotechar='"', header=None,  low_memory=False)

#     # Extrai e divide o cabeçalho manualmente
#     with open(base_path + in_file, 'r', ) as f:
#         header_line = f.readline().strip()
#         header = [col.replace('"','') for col in header_line.split(';')]

#     # Aplica o cabeçalho ao DataFrame
#     df.columns = header
#     # elimina primeira linha que ficou igual ao header
#     df = df.loc[1:,:]

#     filtro = df.TIPOBITO == '2' # obitos não fetais
#     # remove junto as colunas não necessárias
#     if (in_file == 'Mortalidade_Geral_2023.csv') | (in_file == 'Mortalidade_Geral_2024.csv'):
#         df['CONTADOR'] = np.nan
#     df = df.loc[filtro, cols]

#     # Converte a coluna DTOBITO para o formato dd/mm/aaaa
#     df['DTOBITO'] = pd.to_datetime(df['DTOBITO'], format='%d%m%Y', errors='coerce')
#     df['DTNASC'] = pd.to_datetime(df['DTNASC'], format='%d%m%Y', errors='coerce')



#     display(df.shape)
#     df.to_parquet(base_path+parquet_file)

In [12]:
# df

In [13]:
# cols = ['ORIGEM', 'TIPOBITO', 'DTOBITO', 'DTNASC',
#  'IDADE', 'SEXO', 'RACACOR', 'LOCOCOR', 'CIRURGIA',
#  'CAUSABAS', 'CB_PRE', 'CIRCOBITO', 'ACIDTRAB', 'FONTE',
#  'CAUSABAS_O', 'TPOBITOCOR', 'FONTES', 'CONTADOR']

# df = df.loc[:,cols]

In [14]:
# arquivos = glob.glob(base_path + '*resumido*.parquet')
# df_unificado = pd.DataFrame()
# for in_file in arquivos:
#     print(in_file)
#     df = pd.read_parquet(in_file, engine='pyarrow')
    
#     df['DTOBITO'] = pd.to_datetime(df['DTOBITO'], format='%d%m%Y', errors='coerce')
#     df['DTNASC'] = pd.to_datetime(df['DTNASC'], format='%d%m%Y', errors='coerce')

#     df['ANO_OBITO'] = df.DTOBITO.dt.year
#     df['MES_OBITO'] = df.DTOBITO.dt.month
#     print(df.ANO_OBITO.unique())
   
#     df_unificado = pd.concat([df_unificado,df], ignore_index=True)
#     print(df_unificado.ANO_OBITO.unique())

# df_unificado.to_parquet(base_path+'Mortalidade_Geral_2016-2024_resumido.parquet')
# df = df_unificado.copy()
# del df_unificado

# Carrega os dados unificados e filtra dados de interesse

In [15]:
# df = pd.read_parquet(base_path+'Mortalidade_Geral_2016-2024_resumido.parquet', engine='pyarrow')
# # retira os dados de 2024 pois estão incompletos e geram distorções
# filtro = df.ANO_OBITO != 2024
# df = df.loc[filtro, :]

In [16]:
# # remove casos: 1 – acidente; 2 – suicídio; 3 – homicídio; 
# # demais ficam mantidos:4 – outros; 9 – ignorado
# filtro = (df.CIRCOBITO != '1') & (df.CIRCOBITO != '2') & (df.CIRCOBITO != '3')
# df = df.loc[filtro, :]

In [17]:
# # remove casos de acidente de trabalho: 
# filtro = (df.ACIDTRAB != '1') 
# df = df.loc[filtro, :]

In [18]:
# # remove casos:  1 – ocorrência policial;
# # mantido:  2 – hospital; 3 – família; 4 – outra; 9 – ignorado
# filtro = (df.FONTE != '1') 
# df = df.loc[filtro, :]

In [19]:
# # Contar os óbitos por mês e ano
# df_obitos = df.groupby(['ANO_OBITO', 'MES_OBITO']).size().reset_index(name='NUM_OBITOS')

# # Ordena por ano e mês
# df_obitos = df_obitos.sort_values(by=['ANO_OBITO', 'MES_OBITO'])

In [20]:
# # Contar os óbitos por dia
# obitos_por_dia = df.groupby('DTOBITO').size().reset_index(name='NUM_OBITOS')

# # Opcional: Ordenar por data
# obitos_por_dia = obitos_por_dia.sort_values(by='DTOBITO')


In [21]:
# df_obitos.to_parquet(base_path+'obitos_mensais_2016-2023_resumo.parquet')
# obitos_por_dia.to_parquet(base_path+'obitos_diarios_2016-2023_resumo.parquet')

In [22]:
df_obitos = pd.read_parquet(base_path+'obitos_mensais_2016-2023_resumo.parquet')
obitos_por_dia = pd.read_parquet(base_path+'obitos_diarios_2016-2023_resumo.parquet')

# Gráficos

### Exibe a série temporal dos óbitos diários

Os dados entre jan/2016 e dez/2020 mostram uma sazonalidade inesperada... pode-se explorar melhor ao final.

In [23]:
plotar_serie_temporal_obitos_bokeh(obitos_por_dia,width=1000, height=400)

Loading BokehJS ...

# Faz análise de decomposição da série temporal

### Código para decomposição sazonal e verificação de normalidade dos resíduos

In [24]:
# CONVERTE 'DTOBITO' PARA DATETIMEINDEX
df_ts = obitos_por_dia.copy()
df_ts = df_ts.set_index('DTOBITO')

#Salvar a decoposição em result
decomposicao_sazional = seasonal_decompose(df_ts, period=365)
# Retrieve the decomposed components
trend = decomposicao_sazional.trend#.to_timestamp()
seasonal = decomposicao_sazional.seasonal#.to_timestamp()
residual = decomposicao_sazional.resid#.to_timestamp() 

filtro = df_ts.index <'2025-12-31'
#Salvar a decoposição em result
decomposicao_sazional = seasonal_decompose(df_ts.loc[filtro,:], period=30)
# Retrieve the decomposed components
trend = decomposicao_sazional.trend#.to_timestamp()
seasonal = decomposicao_sazional.seasonal#.to_timestamp()
residual = decomposicao_sazional.resid#.to_timestamp() 

### Resultado

In [25]:
plotar_decomposicao_temporal_bokeh(decomposicao_sazional, trend, seasonal, residual, 1200, 160)

Loading BokehJS ...

### Análise da normalidade dos resíduos

In [26]:
verificar_normalidade_residuo(residual)


Teste de Kolmogorov-Smirnov:
Estatística=0.0458, p-valor=0.0000
Resíduo provavelmente NÃO segue uma distribuição normal (rejeita H0).

QQ-plot dos resíduos:


Loading BokehJS ...

# Modelagem dos dados pré COVID-19

### Cria a janela de tempo para o modelo

In [27]:
# coloca a data inicial antes do inicio do banco de dados e a data de fim em 2019
filtro = (obitos_por_dia.DTOBITO > '2010-12-31') & (obitos_por_dia.DTOBITO < '2020-01-01')
df_pre_covid = obitos_por_dia.loc[filtro, :].copy()
# plotar_serie_temporal_obitos_bokeh(df_pre_covid)

### Faz a modelagem dos dados pré COVID-19

In [28]:
# Supondo que df_pre_covid já está carregado e tem as colunas 'DTOBITO' e 'NUM_OBITOS'
# Se DTOBITO não for datetime ainda:
df_pre_covid['DTOBITO'] = pd.to_datetime(df_pre_covid['DTOBITO'])

# Para o ajuste, converta 'DTOBITO' em dias desde o início da série
t0 = df_pre_covid['DTOBITO'].min()
df_pre_covid['dias'] = (df_pre_covid['DTOBITO'] - t0).dt.days

# Extraia os arrays para ajuste
t = df_pre_covid['dias'].values
y = df_pre_covid['NUM_OBITOS'].values

# Modelo sazonal básico (um ciclo anual)
def modelo_sazonal(t, a, b, c, d):
    return a + b * np.sin(2 * np.pi * t / 365.25 + c) + d * np.cos(2 * np.pi * t / 365.25)

# Ajuste dos parâmetros
popt, pcov = curve_fit(modelo_sazonal, t, y)

# Parâmetros ajustados
a, b, c, d = popt
print(f'Parâmetros: a={a:.2f}, b={b:.2f}, c={c:.2f}, d={d:.2f}')

# Geração da curva ajustada
y_fit = modelo_sazonal(t, *popt)
df_pre_covid['y_fit'] = y_fit.astype(int)


Parâmetros: a=3252.13, b=-44.78, c=-4.24, d=-180.35


### Resultado da modelagem dos dados pré COVID-19

In [29]:
grafico_bokeh_duplo(
    df_pre_covid,
    x_col='DTOBITO',
    y1_col='NUM_OBITOS',
    y2_col='y_fit', 
    titulo="Modelagem Sazonal aos Dados Pré COVID-19", 
    y1_legend = "Dados Reais",
    y2_legend = "Dados Modelados",
    y1_color= "navy",
    y2_color="red",
    notebook=True)


Loading BokehJS ...

### Calcula o número de óbitos ajustados subtraindo o modelo sazonal do número real de óbitos

In [30]:
# Extraia os arrays para ajuste
t0 = obitos_por_dia['DTOBITO'].min()
obitos_por_dia['dias'] = (obitos_por_dia['DTOBITO'] - t0).dt.days

t_all = obitos_por_dia['dias'].values
y_all = obitos_por_dia['NUM_OBITOS'].values

# Geração da curva ajustada
y_fit = modelo_sazonal(t_all, *popt)

df_covid = obitos_por_dia.copy()
df_covid['NUM_OBITOS_AJUSTADO'] =  y_all - y_fit
df_covid['MODELO'] = y_fit

### Resultado indica os óbitos por efeito da COVID-19

Após o ajuste dos dados, o número de óbitos antes da COVID-19 tem valor médio próximo de zero, indicando que o modelo sazonal capturou bem a variação natural dos óbitos. 
Já durante o período da pandemia, observa-se um aumento significativo nos óbitos, refletindo o impacto da COVID-19 na mortalidade.

In [31]:
grafico_bokeh_duplo(
    df_covid,
    x_col='DTOBITO',
    y1_col='NUM_OBITOS',
    y2_col='NUM_OBITOS_AJUSTADO', 
    titulo="Ajuste Sazonal aos Dados", 
    y1_legend = "Número de óbitos diários",
    y2_legend = "Número de óbitos modelado",
    notebook=True)


Loading BokehJS ...

### Determinação do momento exato onde ocorre a aceleração dos óbitos

In [32]:
# Determinação do momento exato onde ocorre a aceleração dos óbitos
# Utilizando a derivada do modelo ajustado
# Calcula a derivada numérica do modelo ajustado e um fator de escala
df_covid['grad_mortes'] = np.gradient(trend) *20 / np.gradient(t_all)
filtro = (df_covid.DTOBITO > '2020-03-01') & (df_covid.DTOBITO < '2020-06-01')

grafico_bokeh_duplo(
    df_covid.loc[filtro,:],
    x_col='DTOBITO',
    y1_col='grad_mortes',
    y2_col='NUM_OBITOS_AJUSTADO', 
    titulo="Determinação do Início da Aceleração dos Óbitos", 
    y1_legend = "Dados Ajustados",
    y2_legend = "Taxa de Variação de Mortes",
    notebook=True)


# Determinação do momento onde os óbitos começam a desacelerar
df_covid['grad_mortes'] = np.gradient(trend) *20 / np.gradient(t_all)
filtro = (df_covid.DTOBITO > '2022-01-01') & (df_covid.DTOBITO < '2022-10-01')

grafico_bokeh_duplo(
    df_covid.loc[filtro,:],
    x_col='DTOBITO',
    y1_col='grad_mortes',
    y2_col='NUM_OBITOS_AJUSTADO', 
    titulo="Determinação do fim do transitório dos Óbitos", 
    y1_legend = "Dados Ajustados",
    y2_legend = "Taxa de Variação de Mortes",
    notebook=True)



Loading BokehJS ...

Loading BokehJS ...

Por análise visual da Taxa de Variação de Mortes, o ponto de acentuação das mortes ocorre em 03/04/2020. Para determinar o final do período de mortes intensas, foi verificada a estabilização da cauda de oscilação da taxa de variação das mortes e a curva de dados voltando à sazonalidade pré pandêmica. A data de finalização foi determinada como  08/08/2022.


Valor ajustado é o valor real de mortes reduzido pelo valor esperado se não houvesse pandemia;
Esse valor esperado é calculado pela extrapolação da curva de mortes entre 01/01/2016 e 31/12/2019.
Ao fazer essa subtração, o número de mortes restante é o resultado direto e indireto da pandemia. Ao somar todas as mortes diárias, o resultado é o efeito da pandemia.
Se somarmos as mortes sem o ajuste, teremos o total de mortes do período e a diferença entre os dois mostra a severidade da pandemia.

## Visualização dos dados no período da pandemia

In [33]:
filtro = (df_covid['DTOBITO'] >= '2020-04-15') & (df_covid['DTOBITO'] <= '2022-08-08')

grafico_bokeh_duplo(df_covid.loc[filtro,:], 
                    x_col='DTOBITO', 
                    y1_col = 'NUM_OBITOS',
                    y2_col='NUM_OBITOS_AJUSTADO',
                    y1_legend = 'Número de óbitos diários',
                    y2_legend = 'Número de óbitos modelado',
                    hachura=True)


Loading BokehJS ...

### Cálculo do número de óbitos causados direta ou indiretamente pelo COVID-19

In [50]:
filtro = (df_covid['DTOBITO'] >= '2020-04-15') & (df_covid['DTOBITO'] <= '2022-08-08')

num_mortes_covid = int(df_covid.loc[filtro,'NUM_OBITOS_AJUSTADO'].sum())
num_mortes_periodo_covid = int(df_covid.loc[filtro,'NUM_OBITOS'].sum())

taxa_aumento = int(np.round(100* num_mortes_covid/(num_mortes_periodo_covid - num_mortes_covid),0))

print(f"Número de mortes causadas pela pandemia: {formatar_numero_br(num_mortes_covid)}")
print(f"Número de mortes estimado se não houvesse pandemia: {formatar_numero_br(num_mortes_periodo_covid - num_mortes_covid)}")
print(f"Número de mortes no período da pandemia: {formatar_numero_br(num_mortes_periodo_covid)}")
print(f"Efeito da COVID-19 no número de ótibos: aumento de {taxa_aumento}%")


Número de mortes causadas pela pandemia: 909.108
Número de mortes estimado se não houvesse pandemia: 2.772.135
Número de mortes no período da pandemia: 3.681.243
Efeito da COVID-19 no número de ótibos: aumento de 33%


0.32794506761034364

# Detalhamento das mortes nos meses de inverno

In [35]:
df_2019 = pd.DataFrame()
filtro = (trend.index > '2016-12-31') & (trend.index < '2020-01-01')
df_2019 = trend.loc[filtro]
df_2019 = df_2019.rename('NUM_OBITOS').to_frame()
df_2019.reset_index(inplace=True)

plotar_serie_temporal_obitos_bokeh(df_2019)


Loading BokehJS ...

In [36]:
filtro = (df_2019['DTOBITO'] >= '2017-01-01') & (df_2019['DTOBITO'] <= '2017-12-31')
max_local = df_2019.loc[filtro, 'NUM_OBITOS'].max()
min_local = df_2019.loc[filtro, 'NUM_OBITOS'].min()
print(max_local, min_local, max_local/min_local)

3577.0500000000006 2953.5166666666673 1.2111155628036634


In [37]:
filtro = (df_2019['DTOBITO'] >= '2018-01-01') & (df_2019['DTOBITO'] <= '2018-12-31')
max_local = df_2019.loc[filtro, 'NUM_OBITOS'].max()
min_local = df_2019.loc[filtro, 'NUM_OBITOS'].min()
print(max_local, min_local, max_local/min_local)

3544.333333333333 3002.333333333333 1.1805262573553903


In [38]:
filtro = (df_2019['DTOBITO'] >= '2019-01-01') & (df_2019['DTOBITO'] <= '2019-12-31')
max_local = df_2019.loc[filtro, 'NUM_OBITOS'].max()
min_local = df_2019.loc[filtro, 'NUM_OBITOS'].min()
print(max_local, min_local, max_local/min_local)

3718.15 3126.25 1.1893322670931628
